## ĐẠI HỌC KHOA HỌC TỰ NHIÊN - KHOA CÔNG NGHỆ THÔNG TIN 

## Trực quan hoá dữ liệu

### Lab01: Mối quan hệ của dữ liệu

Nhóm 5
1. 1712117 – Nguyễn Huỳnh Thảo Nhi
2. 1712710 – Lê Quang Quí
3. 1712713 – Lê Bá Quyền
4. 1712775 – Nguyễn Lê Trường Thành


### Import  thư viện

In [ ]:
import requests
import pandas as pd   
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from bs4 import BeautifulSoup
import lxml.html as lh
import seaborn as sns

### Crawl Dữ liệu thống kê từng ngày ca nhiễm virus Covid-19 từ tổ chức Worldometer

In [ ]:
URL = 'https://www.worldometers.info/coronavirus/' #the website the data is extracted
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

In [ ]:
table = soup.find(id='nav-tabContent')
table = table.find(id = 'nav-today')
table = table.find(id = '')
table = table.find(id = 'main_table_countries_today')

In [ ]:
table_rows = table.find_all('tr') #finds all the tables with the tag tr in html
l = [] 
for tr in table_rows: 
    row = []
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    if (len(row) == 0 or '\n' in row[1] ):
        continue  
    row_temp = row[:7] + row[8:10]
    l.append(row_temp) 

dataset = pd.DataFrame(l, columns=["Ranking","Country","Total Cases","New Cases","Total Deaths","New Deaths","Total Recoverd","Active Cases","Serious Cases"])
dataset

### Lưu dữ liệu hàng ngày vào file .csv 

In [ ]:
import datetime
datestr = datetime.date.today().strftime("%Y%m%d")
dataset.to_csv('CoronaData_{}.csv'.format(datestr))
dataset

### Tiền xử lý dữ liệu

In [ ]:
dataset.info()

In [ ]:
# Function to Clean the DataSet
def dataframeCleaner(dataset):
  
    for columnname in dataset: #looping through titles of the table 
        temp = []     
        for column in dataset[columnname]:   #geting column elements for the each title
            column = str(column)
            column = column.replace(',','')# Removing unwanted data clutter
            column = column.replace('+','')#Removing unwanted '+'sign  \
            try:   
                column = int(column)
            except:
                pass
            
            temp.append(column)
        dataset[columnname] = temp
        
 #   dataset = dataset.drop(dataset.tail(5).index)          # Deleting the last row   
    dataset = dataset.replace('N/A', '', regex=True)
    dataset = dataset.replace(r'^\s*$', 0, regex=True)     # Converting empty string to 0
    dataset.replace(['\n'], '', regex=True, inplace=True)
    dataset.replace([','], '', regex=True, inplace=True)
    return dataset

### Dữ liệu sau khi được xử lý

In [ ]:
dataset = dataframeCleaner(dataset) 
dataset[dataset["Country"] == "Asia"]
dataset

In [ ]:
datestr = datetime.date.today().strftime("%Y%m%d")
dataset.to_csv('CoronaData_{}_HorseColic.csv'.format(datestr))
dataset

### Lấy dữ liệu đã lưu về và xử lý vào ngày 10/06/2020

In [ ]:
dataset = pd.read_csv('CoronaData_20200610_HorseColic.csv')
dataset

### Heat map

In [ ]:
df = dataset
df = dataframeCleaner(df)
df = df.drop(df.head(0).index)
df = df.drop(df.tail(8).index)
df

In [ ]:
def heatMap(df):
    corr = df.corr()
    fig, ax = plt.subplots(figsize=(10, 10))
    sns.heatmap(corr,  annot=True, fmt=".2f")
    plt.xticks(range(len(corr.columns)), corr.columns)
    plt.yticks(range(len(corr.columns)), corr.columns)
    plt.show()

In [ ]:
data = df
features = ["Total Cases","New Cases","Total Deaths","New Deaths","Total Recoverd","Active Cases","Serious Cases"]
corr_data = data[features]
heatMap(corr_data)

### Bar Chart

In [ ]:
%matplotlib inline
plt.figure(figsize=(100,50))
corr_data.corr().plot.bar(rot=0, width = 0.8, figsize=(10,10))

#### The lower in each column, least the dependence. for exaple new cases and total recovred are least dependent compaired to new cases vs active cases which signifies that the countries with high active cases will report more new cases where the recovary rate will not effect much for increase of new cases.

### Pie Chart

In [ ]:
labels = ["Total Deaths","Total Recoverd","Active Cases"]
world_total = data[data["Country"] == "World"][labels]
world_data = world_total[labels].values[0].tolist()  # convert ndarray to  list
print(world_total)


In [ ]:

world_series = pd.Series(world_data,index = labels, name = "Total Case of World") 

plot = world_series.plot.pie(figsize = (9,9),fontsize = 18,autopct='%1.1f%%',subplots = False)


### Stack vertical bar chart

In [ ]:
large5 = data[data['Country'] != "World"].nlargest(4,"New Deaths")  #get top 4 country have largest New Deaths
indexs = ["Country","Total Deaths", "Total Recoverd", "Active Cases"]
large_filter = large5[indexs].set_index("Country")
print(large_filter)

In [ ]:
plot = large_filter.plot.bar(stacked=True,figsize = (9,15),rot=0)

### Dot and Line Chart

In [ ]:
total_case_death_reco=df
features = ["Country","Total Cases","Total Deaths","Total Recoverd","Active Cases","Serious Cases"]
df_total_case_death_reco=total_case_death_reco[features]
df_total_case_death_reco = df_total_case_death_reco.drop(df.head(1).index)
df_total_case_death_reco=df_total_case_death_reco.head(10)
df_total_case_death_reco

In [ ]:
#Relationship between Total Case, Total Deaths and Total Recoverd of top 10 

In [ ]:
plt.plot( 'Country', 'Total Cases', data=df_total_case_death_reco, linestyle='-', marker='o')
plt.plot( 'Country', 'Total Deaths', data=df_total_case_death_reco, linestyle='-', marker='o')
plt.plot( 'Country', 'Total Recoverd', data=df_total_case_death_reco, linestyle='-', marker='o')
plt.legend()


### Scatterplot Matrix

In [ ]:
import plotly.express as px
df = px.data.iris()
fig = px.scatter_matrix(df_total_case_death_reco)
fig.show()